In [ ]:
!pip install pycm

     |████████████████████████████████| 71kB 3.8MB/s 
     |████████████████████████████████| 573kB 7.2MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 6.8MB/s 
     |████████████████████████████████| 870kB 15.1MB/s 
     |████████████████████████████████| 3.3MB 34.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=45476e293cd95b6750b23173a4afe60816ab47aab4c64d6405af67360e8fc87a
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
from pycm import *
from transformers import BertTokenizer, BertModel
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import pickle
import sys
from glob import glob  
import math
import shutil
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data.dataset
import torch.utils.data.dataloader
import torchvision.transforms as visionTransforms
import PIL.Image as Image
from torchvision.transforms import ToTensor,ToPILImage

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df=pd.read_csv("path_to_add/MIDAS_Task3/Datasets/Task3_Cleaned_TextModality.csv",index_col=0)

In [ ]:
#Creating the train(0.8) test(0.1) and validation(0.1) splits
dfTrain,dfVal,dfTest=np.split(df.sample(frac=1, random_state=42), [int(.8 * len(df)), int(.9 * len(df))])

In [ ]:
dfTrain=dfTrain.reset_index(drop=True)
dfTest=dfTest.reset_index(drop=True)
dfVal=dfVal.reset_index(drop=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

As visualized in MIDAS_Task3_Preprocessing_Visualization.ipynb there is quite a bit of imbalance in the dataset. We decided to go with the PyTorch WeightedRandomSampler which instead of picking samples randomly in the dataloader will pick it according to weights. And the weight assigned to each class is inversely proportional to the frequency of the class. So the classes with a higher freq are less likely to get sampled, where as classes with a lower frequency are more likely to get sampled

In [ ]:
from torch.utils.data import WeightedRandomSampler
freqLabels=torch.tensor(dfTrain['Label'].value_counts().sort_index(),dtype=torch.double)
weightClass=freqLabels/freqLabels.sum()
weightClass= 1/weightClass
weightClass=(weightClass).tolist()
sampleWeights=[weightClass[i] for i in dfTrain['Label']]
trainSampler=WeightedRandomSampler(sampleWeights,len(dfTrain))

We use a combination of PreTrained BERT, CNNs and Transformer Encoder Blocks as mentioned in the MIDAS_Task3_Detailed_Analysis.pdf

---


We override the PyTorch Dataset class. FlipkartDataset Class gets the product-desc from the dataframe and the respective label. The encoder_plus function from the Transformer's Library is used to encode the given text seq, truncate the seq if it crosses the max_length arg and pad the seq if it is less than the max_length arg. The encode_plus function is also responsible for providing the attention_masks for each seq which helps BERT identify the tokens which are eligible for attention.

The thing that we tried over here was getting the embedding directly from the DataLoader i.e. having pre-trained BERT in the Dataset class itself. But training performance was extremely slow and would often lead to CUDA OOM errors.

In [ ]:
from torch.utils.data import Dataset, DataLoader

class FlipkartDataset(Dataset):

  def __init__(self,dataframe,preTrainedBert,bertTokenizer,maxLength):
    self.data=dataframe
    self.bertTokenizer=bertTokenizer
    self.model=preTrainedBert 
    self.maxLength=maxLength
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    self.productDescription=str(self.data.iloc[idx,0])
    self.label=self.data.iloc[idx,3]

    self.encodedInput=self.bertTokenizer.encode_plus(text=self.productDescription,padding='max_length',truncation="longest_first",max_length=self.maxLength,return_tensors='pt',return_attention_mask=True,return_token_type_ids=True)
    #print(self.encodedInput)
    self.embedding=self.model(input_ids=self.encodedInput['input_ids'],attention_mask=self.encodedInput['attention_mask'],token_type_ids=self.encodedInput['token_type_ids']).last_hidden_state

    return self.embedding,self.label

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
maxLen=0
for i in range(len(df)):
  tempLen=len(word_tokenize(str(df.iloc[i,0])))
  if(tempLen>maxLen):
    maxLen=tempLen
print(maxLen)

1057


We experimented with a couple of maxLength values(128,256,512). But with 256 and 512, there was a considerable increase in the training time. One other reason to choose maxLength=128 was ~70% of the data has seq length of 0-50. Hampering the training performance just to accommodate the complete seq for 30% of the data seemed like an expensive trade-off. Hence the maxLength was taken as 128

In [ ]:
flipkartTrainDataset=FlipkartDataset(dataframe=dfTrain,preTrainedBert=model,bertTokenizer=tokenizer,maxLength=512)
flipkartTestDataset=FlipkartDataset(dataframe=dfTest,preTrainedBert=model,bertTokenizer=tokenizer,maxLength=512)
flipkartValDataset=FlipkartDataset(dataframe=dfVal,preTrainedBert=model,bertTokenizer=tokenizer,maxLength=512)

The Weighted Sampler is just used with the Train Set and not the Test and Validation sets because the model should be capable of recognizing any class at test time irrespective of the frequency of the class

In [ ]:
trainLoader=torch.utils.data.DataLoader(flipkartTrainDataset,batch_size=8,sampler=trainSampler)
testLoader=torch.utils.data.DataLoader(flipkartTestDataset,batch_size=8,shuffle=True)
valLoader=torch.utils.data.DataLoader(flipkartValDataset,batch_size=8,shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
class Encoder_KimCNN(nn.Module):
  def __init__(self,inChannels=1,embeddingDimension=768,numHeads=8,numEncoderLayers=3,numClasses=27):
    super(Encoder_KimCNN,self).__init__()
    self.inChannels=inChannels
    self.embDim=embeddingDimension
    self.numHeads=numHeads
    self.numEncoderLayers=numEncoderLayers
    self.numClasses=numClasses

    self.encoderLayer=nn.TransformerEncoderLayer(d_model=self.embDim,nhead=self.numHeads)
    self.encoderBlock=nn.TransformerEncoder(self.encoderLayer,num_layers=self.numEncoderLayers)
    self.kimConv0=nn.Conv2d(in_channels=self.inChannels,out_channels=100,kernel_size=(2,self.embDim))
    self.kimConv1=nn.Conv2d(in_channels=self.inChannels,out_channels=100,kernel_size=(3,self.embDim))
    self.kimConv2=nn.Conv2d(in_channels=self.inChannels,out_channels=100,kernel_size=(4,self.embDim))
    self.kimConv3=nn.Conv2d(in_channels=self.inChannels,out_channels=100,kernel_size=(5,self.embDim))
    self.dropoutLayer=nn.Dropout(p=0.5)
    self.fc=nn.Linear(400,self.numClasses)

  def forward(self,input):
    input=input.squeeze(dim=1)
    input=input.transpose(1,0)
    
    encoderOutput=self.encoderBlock(input)
    encoderOutput=encoderOutput.transpose(1,0)
    
    kimInput=encoderOutput.unsqueeze(1)
    
    conv0_Output=F.relu(self.kimConv0(kimInput)).squeeze(3)
    conv1_Output=F.relu(self.kimConv1(kimInput)).squeeze(3)
    conv2_Output=F.relu(self.kimConv2(kimInput)).squeeze(3)
    conv3_Output=F.relu(self.kimConv3(kimInput)).squeeze(3)
    
    conv0_Output=F.max_pool1d(conv0_Output,conv0_Output.size(2))
    conv1_Output=F.max_pool1d(conv1_Output,conv1_Output.size(2))
    conv2_Output=F.max_pool1d(conv2_Output,conv2_Output.size(2))
    conv3_Output=F.max_pool1d(conv3_Output,conv3_Output.size(2))

    kimOutput=torch.cat((conv0_Output.squeeze(dim=2),conv1_Output.squeeze(dim=2),conv2_Output.squeeze(dim=2),conv3_Output.squeeze(dim=2)),dim=1)

    output=self.fc(self.dropoutLayer(kimOutput))

    return output


Adam was used as an optimizer with a learning rate of 10^-4 and Cross Entropy Loss was used as the loss function.

In [ ]:
encoder_kimcnn=Encoder_KimCNN()
encoder_kimcnn.to(device)
softmaxLoss = nn.CrossEntropyLoss()
optimizer = optim.Adam(encoder_kimcnn.parameters(), lr=0.0001)

In [ ]:
def Average(lst): 
    return sum(lst) / len(lst) 

def train_model(model,patience,epochs):


  trainBatchCount=0
  testBatchCount=0

  avgTrainAcc=[]
  avgValidAcc=[]
  trainAcc=[]
  validAcc=[]
  trainLosses=[]
  validLosses=[]
  avgTrainLoss=[]
  avgValidLoss=[]


  for i in range(epochs):

    print("Epoch:",i)

    model.train()
    print("Training.....")
    for batch_idx,(data,targets) in enumerate(trainLoader):

      trainBatchCount=trainBatchCount+1

      #data=data.to(device)
      targets=targets.to(device)

      optimizer.zero_grad()

      scores=model(data)
       
      loss=softmaxLoss(scores,targets)

      loss.backward()

      optimizer.step()

      trainLosses.append(float(loss))

      
      correct=0
      total=0
      total=len(targets)


      predictions=torch.argmax(scores,dim=1)

      if i==0:
        print("Targets:-",targets)
        print("Predictions:-",predictions)


      correct = (predictions==targets).sum()
      acc=float((correct/float(total))*100)

      trainAcc.append(acc)

      if ((trainBatchCount%200)==0):

        print("Targets:-",targets)
        print("Predictions:-",predictions)

        print('Loss: {}  Accuracy: {} %'.format(loss.data, acc))

    model.eval()
    print("Validating.....")
    for data,targets in testLoader:

      testBatchCount=testBatchCount+1

      #data=data.to(device=device)
      targets=targets.to(device=device)

      scores=model(data)

      loss=softmaxLoss(scores,targets)

      validLosses.append(float(loss))

      testCorrect=0
      testTotal=0

      _,predictions=scores.max(1)

      testCorrect = (predictions==targets).sum()
      testTotal=predictions.size(0)

      testAcc=float((testCorrect/float(testTotal))*100)

      validAcc.append(testAcc)

      if ((testBatchCount%200)==0):

        print('Loss: {}  Accuracy: {} %'.format(float(loss), testAcc))
    

    trainLoss=Average(trainLosses)
    validLoss=Average(validLosses)
    avgTrainLoss.append(trainLoss)
    avgValidLoss.append(validLoss)
    tempTrainAcc=Average(trainAcc)
    tempTestAcc=Average(validAcc)
    avgTrainAcc.append(tempTrainAcc)
    avgValidAcc.append(tempTestAcc)

    print("Epoch Number:-",i,"  ","Training Loss:-"," ",trainLoss,"Validation Loss:-"," ",validLoss,"Training Acc:-"," ",tempTrainAcc,"Validation Acc:-"," ",tempTestAcc)

    trainAcc=[]
    ValidAcc=[]
    trainLosses=[]
    validLosses=[]

    #early_stopping(validLoss,model)

    #if early_stopping.early_stop:
      #print("Early Stopping")
      #break

  #model.load_state_dict(torch.load("checkpoint.pt"))

  return model,avgTrainLoss,avgValidLoss,avgTrainAcc,avgValidAcc

Having BERT in the Dataset class led CUDA OOM

In [ ]:
encoder_kimcnn,avgTrainLoss,avgValidLoss,avgTrainAcc,avgValidAcc=train_model(encoder_kimcnn,3,10)

Epoch: 0
Training.....


RuntimeError: ignored

In [ ]:
loss_train = avgTrainLoss
loss_val = avgValidLoss
epochs = range(1,11)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='Validation loss')
plt.title('Training and Validation loss(Pre-Training)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_train = avgTrainAcc
loss_val = avgValidAcc
epochs = range(1,11)
plt.plot(epochs, loss_train, 'g', label='Training Acc')
plt.plot(epochs, loss_val, 'b', label='Validation Acc')
plt.title('Training and Validation Accuracy(Pre-Training)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
def checkClassificationMetrics(loader,model):

  completeTargets=[]
  completePreds=[]

  correct=0
  total=0
  model.eval()

  with torch.no_grad():
    for data,targets in loader:
      data=data.to(device=device)
      targets=targets.to(device=device)

      scores=model(data)
      _,predictions=scores.max(1)

      targets=targets.tolist()
      predictions=predictions.tolist()

      completeTargets.append(targets)
      completePreds.append(predictions)
      #correct += (predictions==targets).sum()
      #total+=predictions.size(0)


    completeTargetsFlattened=[item for sublist in completeTargets for item in sublist]
    completePredsFlattened=[item for sublist in completePreds for item in sublist]

    cm = ConfusionMatrix(actual_vector=completeTargetsFlattened, predict_vector=completePredsFlattened)
    return cm

In [ ]:
softmaxCM=checkClassificationMetrics(testLoader,encoder_kimcnn)